## Описание данных

Имеются данные по 90,189 игрокам, которые установили игру во время A/B теста и случайно были распределены между группами - gate_30 или gate_40
* userid - уникальный идентификатор пользователя.
* version - в какую из групп попал пользователь: (gate_30 - a gate at level 30) или тестовую - (gate_40 - a gate at level 40).
* sum_gamerounds - количество игровых раундов, сыгранных пользователем за 1 неделю после установки
* retention_1 - вернулся ли пользователь в игру на 1й день после установки
* retention_7 - вернулся ли пользователь в игру на 7й день после установки

Различия между группами будут определяться на основе sum_gamerounds

## Изучение данных

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
from scipy.stats import shapiro
import scipy.stats as stats
import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
data = pd.read_csv('/datasets/cookie_cats.csv')

In [49]:
data

,userid,version,sum_gamerounds,retention_1,retention_7,retention
0,116,A,3,False,False,0
1,337,A,38,True,False,0
2,377,B,165,True,False,0
3,483,B,1,False,False,0
4,488,B,179,True,True,1
...,...,...,...,...,...,...
90184,9999441,B,97,True,False,0
90185,9999479,B,30,False,False,0
90186,9999710,A,28,True,False,0
90187,9999768,B,51,True,False,0


In [4]:
# проверка количества уникальных пользователей

data.shape[0] == data['userid'].nunique()

True

In [5]:
# перцентили количества сыгранных раундов

data.describe([0.01, 0.05, 0.10, 0.20, 0.80, 0.90, 0.95, 0.99])[["sum_gamerounds"]].T

,count,mean,std,min,1%,5%,10%,20%,50%,80%,90%,95%,99%,max
sum_gamerounds,90189.0,51.872457,195.050858,0.0,0.0,1.0,1.0,3.0,16.0,67.0,134.0,221.0,493.0,49854.0


In [6]:
# посмотрим на данные по уровням в разрезе тестовой и контрольной группы
data.groupby('version')['sum_gamerounds'].agg(['count', 'median', 'mean', 'std', 'max'])

,count,median,mean,std,max
version,,,,,
gate_30,44700,17,52.456264,256.716423,49854
gate_40,45489,16,51.298776,103.294416,2640


Виден сильный перекос в данных из-за максимального уровня игрока

In [7]:
# уберем максимальное значение из ДФ-ма
data = data[data['sum_gamerounds'] < data['sum_gamerounds'].max()]
data.groupby('version')['sum_gamerounds'].agg(['count', 'median', 'mean', 'std', 'max'])

,count,median,mean,std,max
version,,,,,
gate_30,44699,17,51.342111,102.057598,2961
gate_40,45489,16,51.298776,103.294416,2640


Теперь данные по группам практически идентичны

In [8]:
data[data['sum_gamerounds'] == 0].shape[0]

3994

При этом 3994 игрока не сыграли ни разу

In [51]:
data.groupby("sum_gamerounds")['userid'].count().reset_index().head(5)

,sum_gamerounds,userid
0,0,3994
1,1,5538
2,2,4606
3,3,3958
4,4,3629


Распределение игроков по количеству sum_gamerounds

## Подготовка данных

In [10]:
data

,userid,version,sum_gamerounds,retention_1,retention_7
0,116,gate_30,3,False,False
1,337,gate_30,38,True,False
2,377,gate_40,165,True,False
3,483,gate_40,1,False,False
4,488,gate_40,179,True,True
...,...,...,...,...,...
90184,9999441,gate_40,97,True,False
90185,9999479,gate_40,30,False,False
90186,9999710,gate_30,28,True,False
90187,9999768,gate_40,51,True,False


In [36]:
data.groupby(["version", "retention_1"]).sum_gamerounds.agg(["count", "median", "mean", "std", "max"])

count  median       mean         std   max
version retention_1                                            
gate_30 False        24665       6  16.359092   36.528426  1072
        True         20034      48  94.411700  135.037697  2961
gate_40 False        25370       6  16.340402   35.925756  1241
        True         20119      49  95.381182  137.887256  2640

In [31]:
data.groupby(["version", "retention_7"]).sum_gamerounds.agg(["count", "median", "mean", "std", "max"])

count  median        mean         std   max
version retention_7                                             
gate_30 False        36198      11   25.796508   43.316158   981
        True          8501     105  160.117516  179.358560  2961
gate_40 False        37210      11   25.856356   44.406112  2640
        True          8279     111  165.649837  183.792499  2294

In [40]:
# пользователям, которые вернулись на 1й и на 7й день присвоим 1 в столбце retention
data['retention'] = np.where((data.retention_1 == True) & (data.retention_7 == True), 1,0)
data

,userid,version,sum_gamerounds,retention_1,retention_7,retention
0,116,gate_30,3,False,False,0
1,337,gate_30,38,True,False,0
2,377,gate_40,165,True,False,0
3,483,gate_40,1,False,False,0
4,488,gate_40,179,True,True,1
...,...,...,...,...,...,...
90184,9999441,gate_40,97,True,False,0
90185,9999479,gate_40,30,False,False,0
90186,9999710,gate_30,28,True,False,0
90187,9999768,gate_40,51,True,False,0


In [35]:
data.groupby(["version", "retention"])["sum_gamerounds"].agg(["count", "median", "mean", "std", "max"])

count  median        mean         std   max
version retention                                             
gate_30 0          38023      12   28.070273   48.017452  1072
        1           6676     127  183.886309  189.626390  2961
gate_40 0          38983      12   28.103353   48.927850  2640
        1           6506     133  190.282355  194.220077  2294

## A/B-test

In [42]:
# явно укажем группы для игроков
data['version'] = np.where(data['version'] == 'gate_30', 'A', 'B')
data

,userid,version,sum_gamerounds,retention_1,retention_7,retention
0,116,A,3,False,False,0
1,337,A,38,True,False,0
2,377,B,165,True,False,0
3,483,B,1,False,False,0
4,488,B,179,True,True,1
...,...,...,...,...,...,...
90184,9999441,B,97,True,False,0
90185,9999479,B,30,False,False,0
90186,9999710,A,28,True,False,0
90187,9999768,B,51,True,False,0


Проверка результатов A/B-теста:
* проверка на нормальность распределения
* проверка на гомогенность
* выбор критерия проверки гипотезы

In [48]:
def AB_Test(dataframe, group, target):

    # Packages
    from scipy.stats import shapiro
    import scipy.stats as stats

    # разобью ДФ на 2 группы
    groupA = dataframe[dataframe[group] == "A"][target]
    groupB = dataframe[dataframe[group] == "B"][target]

    # проверка на нормальное распределение
    ntA = shapiro(groupA)[1] < 0.05
    ntB = shapiro(groupB)[1] < 0.05
    # H0: Распределение нормальное! - False
    # H1: Распределение не нормальное! - True

    if (ntA == False) & (ntB == False): # "H0: Нормальное распределение"
        # Parametric Test
        # Проверка на гомогенность
        leveneTest = stats.levene(groupA, groupB)[1] < 0.05
        # H0: Гомогенные данные: False
        # H1: Гетерогенные: True

        if leveneTest == False:
            # Если выборка гомогенная
            ttest = stats.ttest_ind(groupA, groupB, equal_var=True)[1]
            # H0: M1 == M2 - False
            # H1: M1 != M2 - True
        else:
            # Если выборка гетерогенная
            ttest = stats.ttest_ind(groupA, groupB, equal_var=False)[1]
            # H0: M1 == M2 - False
            # H1: M1 != M2 - True
    else:
        # Непараметрический тест
        ttest = stats.mannwhitneyu(groupA, groupB)[1]
        # H0: M1 == M2 - False
        # H1: M1 != M2 - True

    # Result
    temp = pd.DataFrame({
        "AB Hypothesis":[ttest < 0.05],
        "p-value":[ttest]
    })
    temp["Test Type"] = np.where((ntA == False) & (ntB == False), "Parametric", "Non-Parametric")
    temp["AB Hypothesis"] = np.where(temp["AB Hypothesis"] == False, "Fail to Reject H0", "Reject H0")
    temp["Comment"] = np.where(temp["AB Hypothesis"] == "Fail to Reject H0", "Между группами нет различий!", "Между группами есть различия!")

    # Columns
    if (ntA == False) & (ntB == False):
        temp["Homogeneity"] = np.where(leveneTest == False, "Yes", "No")
        temp = temp[["Test Type", "Homogeneity","AB Hypothesis", "p-value", "Comment"]]
    else:
        temp = temp[["Test Type","AB Hypothesis", "p-value", "Comment"]]
    return temp



# Apply A/B Testing
AB_Test(dataframe=data, group = "version", target = "sum_gamerounds")

,Test Type,AB Hypothesis,p-value,Comment
0,Non-Parametric,Reject H0,0.025446,Между группами есть различия!


Вывод: между группами есть статистически значимые различия